In [2]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages import BaseMessage
from camel.societies.workforce import Workforce
from camel.toolkits import SearchToolkit
from camel.tasks import Task
from camel.toolkits import FunctionTool

import os

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
     api_key='94f17f1f-3099-4689-83dd-83f8f3698ee8'
)
        
# 创建一个 Workforce 实例
workforce = Workforce(
    description="旅游攻略制作与评估工作组",
    new_worker_agent_kwargs={'model':model},
    coordinator_agent_kwargs={'model':model},
    task_agent_kwargs={'model':model}
)

search_tool = FunctionTool(SearchToolkit().search_duckduckgo)

In [3]:
search_agent = ChatAgent(
            system_message="""你是一个专业的旅游信息搜索助手。你的职责是:
                1. 搜索目的地的主要景点信息
                2. 搜索当地特色美食信息
                3. 搜索交通和住宿相关信息
                请确保信息的准确性和实用性。""",
            model=model,
            tools=[search_tool],
            output_language='中文'
        )

In [4]:
planner_agent = ChatAgent(
            system_message="""你是一个专业的旅行规划师。你的职责是:
                1. 根据景点分布规划合理的游览顺序
                2. 为每天安排适量的景点和活动
                3. 考虑用餐、休息等时间
                4. 注意不同季节的特点
                请确保行程安排合理且具有可行性。""",
            model=model,
            output_language='中文'
        )

In [5]:
reviewer_agent = ChatAgent(
    system_message="""你是一个经验丰富的旅行爱好者。你的职责是:
        1. 从游客角度评估行程的合理性
        2. 指出可能的问题和改进建议
        3. 补充实用的旅行小贴士
        4. 评估行程的性价比
        请基于实际旅行经验给出中肯的建议。""",
    model=model,
    output_language='中文'
)

In [6]:
# 添加一个执行网页搜索的Agent
workforce.add_single_agent_worker(
    "一个能够执行网页搜索的Agent",    worker=search_agent,
)

# 添加工作节点
workforce.add_single_agent_worker(
    "负责搜索目的地相关信息",
    worker=search_agent
).add_single_agent_worker(
    "负责制定详细行程规划",
    worker=planner_agent
).add_single_agent_worker(
    "负责从游客角度评估行程",
    worker=reviewer_agent
)

Workforce 2399449069712 (旅游攻略制作与评估工作组)

from camel.tasks import Task

# 创建一个用于测试的任务
task = Task(
    content="规划一个3天的巴黎旅行计划。",
    id="0",  # id可以是任何标记字符串
)

# 使用jupyter来运行代码请打开下面两行注释
import nest_asyncio
nest_asyncio.apply()

# 让 Workforce 处理这个任务
task = workforce.process_task(task)

print(task.result)

In [ ]:
# 创建一个更详细的任务描述
task = Task(
    content="""
    请规划一个3天的巴黎旅行计划，需要包含以下内容：
    1. 每天的行程安排（上午/下午/晚上）
    2. 主要景点游览顺序
    3. 用餐和休息时间
    4. 交通建议
    
    请确保行程安排合理且实用。
    """,
    id="paris_trip",
    expected_output_format={
        "type": "text",
        "structure": "day_by_day_plan"
    }
)

# 使用jupyter来运行代码
import nest_asyncio
nest_asyncio.apply()

# 添加错误处理
try:
    task = workforce.process_task(task)
    print("任务结果：")
    print(task.result)
except Exception as e:
    print(f"处理任务时出现错误: {str(e)}")
    # 尝试使用简单的串行处理
    try:
        # 获取搜索结果
        search_result = search_agent.step("搜索巴黎三天游的主要景点信息")
        print("\n搜索结果：")
        print(search_result.msg.content)
        
        # 制定行程计划
        plan = planner_agent.step(f"基于以下信息制定三天行程：{search_result.msg.content}")
        print("\n行程计划：")
        print(plan.msg.content)
        
        # 评估行程
        review = reviewer_agent.step(f"评估以下行程：{plan.msg.content}")
        print("\n行程评估：")
        print(review.msg.content)
    except Exception as e:
        print(f"备选方案也失败了: {str(e)}")

2025-03-17 09:29:01,730 - camel.agents.chat_agent - WARNING - Failed to parse response: Certainly! Here's the formatted content:

---

Based on the task requirements and the descriptions of the available worker nodes, the most suitable worker node for this task is:

**<2398765383248>:<负责搜索目的地相关信息>:<tools: search_duckduckgo>**

This worker node is specifically designed to handle tasks related to searching for destination-related information. It aligns perfectly with the task of collecting information about Paris tourism, including must-visit attractions, food recommendations, and cultural activities.

---


JSONDecodeError: Expecting value: line 1 column 1 (char 0)